In [ ]:
import pandas as pd
import pennylane.numpy as np
import pennylane as qml


from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
import tensorflow as tf

# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
dataframe = pd.read_csv(r'4features-final.csv')
dataset = dataframe.values


X=dataset[:,1:5]
Y=dataset[:,5:]

scaler = StandardScaler()
StdScaler=scaler.fit(X)
Stdmean=scaler.mean_
X=scaler.transform(X)

scaler = StandardScaler()
StdScaler2=scaler.fit(Y)
Stdmean2=scaler.mean_
Y=scaler.transform(Y)

months = len(Y)
TRAIN_TEST_SPLIT = 0.78

split = split = int(months * TRAIN_TEST_SPLIT)
TrainX = X[0:split]
TestX = X[split:]

TrainY = Y[0:split]
TestY = Y[split:]


print("Len(Train):",len(TrainX))
print("Len(Test):",len(TestX))

In [ ]:
def H_layer(n_qubits):
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)

def Data_AngleEmbedding_layer(inputs):
    qml.templates.AngleEmbedding(inputs,rotation='Y', wires=range(n_qubits))

def RY_layer(w):
    print(w.shape)
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def ROT_layer(w):
    for i in range(4):
        qml.Rot(*w[i],wires=i)

def strong_entangling_layer(nqubits):
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])
    
def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2): 
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])

In [ ]:
n_qubits = 4
n_layers = 1

dev = qml.device('default.qubit', wires=n_qubits)
@qml.qnode(dev)
def qnode(inputs, weights_1):
    H_layer(n_qubits)
    Data_AngleEmbedding_layer(inputs)
    for k in range(n_layers):
        entangling_layer(n_qubits)
        ROT_layer(weights_1[k])
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
weight_shapes = {"weights_1": (n_layers,n_qubits,3)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)
Activation=tf.keras.layers.Activation(activations.linear)
clayer_2 = tf.keras.layers.Dense(1,kernel_initializer='normal')
model = tf.keras.models.Sequential([qlayer,Activation, clayer_2])
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss="mae")

In [ ]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
re=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min', min_lr=0.00001)
fitting = model.fit(TrainX, TrainY, epochs=30, batch_size=1, validation_split=0.1, callbacks=[re], verbose=1)

In [ ]:
y_predict=model.predict(TestX,verbose=1)
y_predict=y_predict.T
y_predict=y_predict[0]

In [ ]:
newTestY = scaler.inverse_transform(TestY)
newy_predict = scaler.inverse_transform(y_predict.reshape(-1, 1))

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(newTestY, newy_predict))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(newTestY, 'm-', label='Dados Reais')
plt.plot(newy_predict, 'g-', label='Previsão')
plt.title('Experimento 1 com 4 variáveis e 1 camada')
plt.axis([0, 12, 6000, 16000])
plt.xlabel('Mês')
plt.ylabel('Target')
plt.legend()
plt.savefig('Experimento 1 com 4 variáveis e 1 camada')
plt.show()

In [ ]:
newTestY

In [ ]:
newy_predict

In [ ]:
plt.plot(fitting.epoch, fitting.history.get('loss'), 'm-', label='loss')
plt.plot(fitting.epoch, fitting.history.get('val_loss'), 'g-', label='val_loss')
# plt.plot(newy_predict, 'g-', label='Previsão')
plt.title('MAE')
#plt.axis([-2, 10, -10.0, 10.0])
plt.xlabel('Mês')
plt.ylabel('Target')
plt.legend()
plt.show()

In [ ]:
print(fitting.history.get('loss'))

In [ ]:
fitting.history.get('val_loss')